In [1]:
'''
Samuel Grant 2024

Read failure events and make plots. 

Based on PlotFailures.py. Better suited to notebook (so we don't have to load the data each time). 

''' 

'\nSamuel Grant 2024\n\nRead failure events and make plots. \n\n'

In [23]:
# External libraries
import numpy as np
import pandas as pd
import awkward as ak
import uproot

# Internal libraries
sys.path.append(os.path.abspath("../PyMacros"))
import Utils as ut
from Mu2eEAF import ReadData as rd 

In [16]:
# Get file list
fileListPath="../Txt/FileLists/MDC2020aeOnExpData.txt"
def ReadFileList(fileListPath):
  with open(fileListPath, "r") as fileList_:
    lines = fileList_.readlines()
    lines = [line.strip() for line in lines]  # Remove leading/trailing whitespace
  return lines
fileList_=ReadFileList(fileListPath)
print(f"---> {len(fileList_)} files in list.")

---> 96 files in list.


In [68]:
'''
Get failure info
For each file, collect the events. 

This takes quite a while (2-3 minutes maybe).
I spent a full day trying to multithread this, it blows up my memory. 
'''
def GetFailures(fileList_, PEs=10, layers="3", particle="all", cut="singles"):

     # Collect failure info
    failureFilePath = f"../Txt/MDC2020ae/concatenated/failures_concise/failures_concise_all_{PEs}PEs{layers}Layers_{cut}.csv"

    print(f"\n---> Using info file {failureFilePath}.")
    failureInfo_ = pd.read_csv(failureFilePath)

    # Collect file list
    tags_ = list(set(failureInfo_["Tag"]))
    
    # Extract the tag from the file name
    def ExtractTag(fileName):
        parts = fileName.split('.')
        if len(parts) > 1:
            return parts[-2]
        return None

    # Filter and sort files based on tags
    fileList_ = sorted(
        [file for file in fileList_ if ExtractTag(file) in tags_]
        , key=lambda file: tags_.index(ExtractTag(file))
    )

    # testing
    # tags_ = tags_[:1]
    # fileList_ = fileList_[:1]
    
    # Bug check
    if len(fileList_) == len(tags_):
        print("\n---> Collected and sorted failure file names.")
    else:
        raise Exception("\n---> len(fileList_) != len(tags_)")

    completedFiles = 0
    totalFiles = len(fileList_)

    print(f"\n---> Iterating through {len(fileList_)} files.\n") 

    # Master array to hold results
    data_ = ak.Array([])
    
    # Iterate through file list.
    for tag, fileName in zip(tags_, fileList_): 
    
        # Find failure events
        thisFailureInfo_ = failureInfo_[failureInfo_["Tag"] == tag]
        outputStr = ( 
            f"\n--->\n" 
            f"fileName: {fileName}\n"
            f"tag: {tag}\n"
            f"failures:\n{thisFailureInfo_}\n"
            f"---"
        )
        if False: print(outputStr)
    
        # Read the file
        with uproot.open(fileName) as file: 
            
            # file = (fileName, quiet=True)
            # Get array
            thisData_ = ut.GetData(file)
        
            if False: print(f"\n---> Loaded corresponding data.\n{thisData_}")
        
            if False: print(f"\n---> Applying masks.")
        
            # Extract unique values from DataFrame
            runs_ = set(thisFailureInfo_["evtinfo.run"])
            subruns_ = set(thisFailureInfo_[" evtinfo.subrun"])
            events_ = set(thisFailureInfo_[" evtinfo.event"])
        
            # Construct masks
            runCondition = ak.any([thisData_["evt"]["evtinfo.run"] == value for value in runs_], axis=0)
            subrunCondition = ak.any([thisData_["evt"]["evtinfo.subrun"] == value for value in subruns_], axis=0)
            eventCondition = ak.any([thisData_["evt"]["evtinfo.event"] == value for value in events_], axis=0)
        
            # Apply masks
            thisData_ = thisData_[runCondition & subrunCondition & eventCondition]
        
            # Append to master array
            if False: print(f"\n---> Appending failures to master array.")
            data_ = ak.concatenate([data_, thisData_], axis=0)

            if False:
                print(thisFailureInfo_)
                ut.PrintNEvents(data_)
    
            completedFiles += 1
            percentComplete = (completedFiles / totalFiles) * 100
            
            print(f"\r---> Processed {fileName} ({percentComplete:.1f}% complete...)", end="") 

    inputEventList = failureInfo_[" evtinfo.event"]
    outputEventList = ak.flatten(data_["evt"]["evtinfo.event"], axis=None)

    return data_
    
    # # print(outputEventList)
    # # ut.PrintNEvents(data_)

    # if len(inputEventList) == len(outputEventList):
    #     print("data_ contains the correct number of events!")
    #     print("Done!")
    #     print(len(inputEventList), len(outputEventList))
    #     return data_
    # else:
    #     raise ValueError(f"data_ contains the wrong number of events! {len(outputEventList)} compared with {len(inputEventList)}.")

In [21]:
# Parameters
PEs=10
layers="3"
particle="all"

In [69]:
data_singles_ = GetFailures(fileList_, cut="singles")


---> Using info file ../Txt/MDC2020ae/concatenated/failures_concise/failures_concise_all_10PEs3Layers_singles.csv.

---> Collected and sorted failure file names.

---> Iterating through 36 files.

---> Processed /exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00058/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000025.root (100.0% complete...)

In [71]:
ut.PrintNEvents(data_singles_, 1)

-------------------------------------------------------------------------------------
***** evt *****
evtinfo.run: 1205
evtinfo.subrun: 7468
evtinfo.eventid: 384774
***** crv *****
crvcoincs.sectorType: [2, 3]
crvcoincs.nLayers [2, 4]
crvcoincs.angle: [-0.521, -0.431]
crvcoincs.pos.fCoordinates: ([-768, -535], [4.64e+03, 4.91e+03], [-437, -727])
crvcoincs.timeStart: [4.46e+04, 4.46e+04]
crvcoincs.time: [4.46e+04, 4.46e+04]
crvcoincs.PEs: [130, 767]
crvcoincs.PEsPerLayer[4]: [[0, 51.3, 0, 78.6], [351, 149, 133, 135]]
crvcoincs.nHits: [6, 20]
crvcoincsmc.pdgId: [-11, -11]
crvcoincsmc.valid: [True, True]
***** trk *****
kl.status: [1]
kl.nactive: [9]
kl.nhits: [11]
kl.nplanes: [2]
kl.nnullambig: [4]
kl.ndof: [13]
kl.kl.fitcon: [0.657]
***** trkfit *****
klfit: [[{mom: {fCoordinates: {...}}, pos: {...}, time: 4.46e+04, ...}, ..., {...}]]
klfit.sid: [[0, 2, 4, 4, 200, 200, 200]]
klfit.sindex: [[0, 0, 0, 0, 0, 1, 2]]
klkl: [[{d0: -545, phi0: -1.87, z0: 1.47e+03, theta: 1.24, mom: 2e+03, ...}

In [72]:
data_singles_track_cuts_ = GetFailures(fileList_, cut="singles_track_cuts")


---> Using info file ../Txt/MDC2020ae/concatenated/failures_concise/failures_concise_all_10PEs3Layers_singles_track_cuts.csv.

---> Collected and sorted failure file names.

---> Iterating through 9 files.

---> Processed /exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00034/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000051.root (100.0% complete...)

In [73]:
ut.PrintNEvents(data_singles_track_cuts_, 1)

-------------------------------------------------------------------------------------
***** evt *****
evtinfo.run: 1205
evtinfo.subrun: 5349
evtinfo.eventid: 183839
***** crv *****
crvcoincs.sectorType: [2, 3]
crvcoincs.nLayers [2, 2]
crvcoincs.angle: [0, 0.334]
crvcoincs.pos.fCoordinates: ([749, 1.19e+03], [4.63e+03, 4.96e+03], [-437, -230])
crvcoincs.timeStart: [7.88e+04, 7.88e+04]
crvcoincs.time: [7.88e+04, 7.88e+04]
crvcoincs.PEs: [164, 275]
crvcoincs.PEsPerLayer[4]: [[0, 61, 103, 0], [0, 0, 105, 170]]
crvcoincs.nHits: [4, 8]
crvcoincsmc.pdgId: [11, 11]
crvcoincsmc.valid: [True, True]
***** trk *****
kl.status: [1]
kl.nactive: [16]
kl.nhits: [16]
kl.nplanes: [4]
kl.nnullambig: [2]
kl.ndof: [27]
kl.kl.fitcon: [0.425]
***** trkfit *****
klfit: [[{mom: {fCoordinates: {...}}, pos: {...}, time: 7.88e+04, ...}, ..., {...}]]
klfit.sid: [[0, 2, 4, 4, 200, 200, 200]]
klfit.sindex: [[0, 0, 0, 0, 0, 1, 2]]
klkl: [[{d0: 239, phi0: -2.57, z0: 860, theta: 1.11, mom: 2e+03, t0: ..., ...}, ...]]
k

In [ ]:
# Now we have the failure data...
# Plot it here...

In [75]:
# What is more interesting is those that pass the singles cut, but not the track cut.
# Find the unique failures. Although, this will be biased a bit because sometimes the track cuts generate an exception, while the singles do not!
# Need to ID a file where this doesn't happen. I think 00000000 is good. 
print(fileList)

['/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00033/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000080.root', '/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00094/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000048.root', '/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00034/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000051.root', '/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00061/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000056.root', '/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00050/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000035.root', '/exp/mu2e/data/users/sgrant/